## What Does this Example Do?
This project should load data from a CSV hosted from 'somewhere' and update/create new records in Airtable 

In [70]:
import os
import pandas as pd
import pyairtable # to interact with Airtable METADATA API
from pyairtable import Table, Api, Base, metadata # to interact with Airtable REST API
from dotenv import load_dotenv # to load .env files with environment variables
import yaml # to read config files
import requests
from requests.auth import HTTPBasicAuth

# Load .env files
load_dotenv()

# Read Config File
with open("config.yaml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)
    print("Config Read successful")

Config Read successful


## Get data from your source
- Authenticate with source
- Get data
- Transform into JSON

In [71]:
# Work with DataSource team to navigate authentication with DataSource
username = "username"
password = "password"

 # Download File ()
csv_url = 'https://raw.githubusercontent.com/fivethirtyeight/data/master/comic-characters/marvel-wikia-data.csv'
df = pd.read_csv(csv_url)

# Convert to JSON
df.to_json("data.json", indent =4, orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)
data = pd.read_json("data.json")

# Rename Header Row with Name from Config File
rename_dict = config[0]['FieldNames']
renamed_data = data.rename(rename_dict, axis =1)

# Remove Null / NA Values
prepped_data = renamed_data.fillna({"GSM":"", "Year of First Appearance":0}) # Change this line to reflect your dataset
prepped_data_dict = prepped_data.to_dict('records')


## Update Values in Airtable
- [Example Base](https://airtable.com/invite/l?inviteId=inv0QZYyRSLsRVYGv&inviteToken=09a62e5c72983bc5245f6b34b78fc11d5da7f0794ac3816d190a7dfbd469bf2e&utm_source=email)
- Using typecast = True, will allow you to easily create new Select Options and Linked Records when needed.

In [72]:
# Configuration variables for Airtable
AIRTABLE_API_KEY = os.environ['AIRTABLE_API_KEY']
AIRTABLE_BASE_ID = os.environ['AIRTABLE_BASE_ID']
AIRTABLE_TABLE_ID = os.environ['AIRTABLE_TABLE_ID']
AIRTABLE_UNIQUE_FIELD_NAME = os.environ['AIRTABLE_UNIQUE_FIELD_NAME']

# Initialize Airtable client
api = Api(AIRTABLE_API_KEY)
Table = Table(AIRTABLE_API_KEY, AIRTABLE_BASE_ID, AIRTABLE_TABLE_ID)

# Define input records(from the source system). This would usually be an API call or reading in a CSV or other format of data.
inputRecords = prepped_data_dict

# Retrieve all existing records from the base through the Airtable REST API
allExistingRecords = Table.all()
print('{} existing records found'.format(len(allExistingRecords)))

# Create an object mapping of the primary field to the record ID
# Remember, it's assumed that the AIRTABLE_UNIQUE_FIELD_NAME field is truly unique
upsertFieldValueToExistingRecordId = {
    existingRecord['fields'].get(AIRTABLE_UNIQUE_FIELD_NAME): existingRecord['id'] for existingRecord in allExistingRecords
}

# Create two arrays: one for records to be created, one for records to be updated
recordsToCreate = []
recordsToUpdate = []

# For each input record, check if it exists in the existing records. If it does, update it. If it does not, create it.
print('\nProcessing {} input records to determine whether to update or create'.format(
    len(inputRecords)))
for inputRecord in inputRecords:
    recordUniqueValue = inputRecord.get(AIRTABLE_UNIQUE_FIELD_NAME, None)
    print('\tProcessing record w / \'{}\' === \'{}\''.format(
        AIRTABLE_UNIQUE_FIELD_NAME, recordUniqueValue))

    existingRecordIdBasedOnUpsertFieldValueMaybe = upsertFieldValueToExistingRecordId.get(
        recordUniqueValue)

    # and if the upsert field value matches an existing one...
    if existingRecordIdBasedOnUpsertFieldValueMaybe:
        # Add record to list of records to update
        print('\t\tExisting record w / ID {} found; adding to recordsToUpdate'.format(
            existingRecordIdBasedOnUpsertFieldValueMaybe))
        recordsToUpdate.append(
            dict(id=existingRecordIdBasedOnUpsertFieldValueMaybe, fields=inputRecord))
    else:
        # Otherwise, add record to list of records to create
        print('\t\tNo existing records match; adding to recordsToCreate')
        recordsToCreate.append(inputRecord)

# Read out array sizes
print("\n{} records to create".format(len(recordsToCreate)))
print("{} records to update".format(len(recordsToUpdate)))

# Perform record creation - typecast = True to crete new Single Select Options
Table.batch_create(recordsToCreate, typecast=True)

# Perform record updates on existing records - typecast = True to crete new Single Select Options
Table.batch_update(recordsToUpdate, typecast=True)

print("\n\nScript execution complete!")

6890 existing records found

Processing 16376 input records to determine whether to update or create
	Processing record w / 'Page ID' === '1678'
		Existing record w / ID reckoaPutjduwmRlj found; adding to recordsToUpdate
	Processing record w / 'Page ID' === '7139'
		Existing record w / ID recFFNnWdC4EZ5ChP found; adding to recordsToUpdate
	Processing record w / 'Page ID' === '64786'
		Existing record w / ID rec77rx6dMZkW2RBp found; adding to recordsToUpdate
	Processing record w / 'Page ID' === '1868'
		Existing record w / ID rec7cjqdYKs4hoesf found; adding to recordsToUpdate
	Processing record w / 'Page ID' === '2460'
		Existing record w / ID reco51ggkSssRIuw6 found; adding to recordsToUpdate
	Processing record w / 'Page ID' === '2458'
		Existing record w / ID rece2yLcWRc7LudIG found; adding to recordsToUpdate
	Processing record w / 'Page ID' === '2166'
		Existing record w / ID reckDzscZAeOANv6I found; adding to recordsToUpdate
	Processing record w / 'Page ID' === '1833'
		Existing rec

KeyboardInterrupt: 

## Compare information from DataSource and Airtable